In [4]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
import sys
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.model_selection import train_test_split

In [5]:
import pandas as pd
train_labels = pd.read_csv('../data/train_labels.csv')
train_features = pd.read_csv('../data/train_features.csv')

train_merged = pd.merge(train_labels, train_features, on='id')
train_merged_back = train_merged.copy()

#print(train_merged.head())

In [6]:
unique_sites = train_merged['site'].unique()

temp_df = train_merged.groupby("site").sum()
temp_df.drop("id", axis=1, inplace=True)
temp_df.drop("filepath", axis=1, inplace=True)
#print(temp_df)
total_sightings = temp_df.iloc[:, 1:].sum(axis=1).tolist()

unique_sites = pd.DataFrame({"site": unique_sites, "total_sightings": total_sightings})

unique_sites = unique_sites.sort_values(by="total_sightings", ascending=False)

#for site in unique_sites['site']:
#    print(f'"{site}",', end='')




In [7]:
def get_data(sites):
    processing = train_merged_back.loc[train_merged_back['site'].isin(sites)]
    print(processing.shape)
    labels = processing.drop(['filepath', 'id', 'site'], axis=1)
    images = []

    for index, row in processing.iterrows():  
        image_path = row['filepath']
        image = cv.imread(os.path.join('../data/', image_path))
        image = cv.resize(image, (960, 540))

        images.append(image)
        labels = labels.loc[labels.index.repeat(len(labels) + 1)].reset_index(drop=True)
        labels.loc[len(labels)] = row 

        flipped_image = cv.flip(image.copy(), 0)
        images.append(flipped_image)
        labels = labels.loc[labels.index.repeat(len(labels) + 1)].reset_index(drop=True)
        labels.loc[len(labels)] = row 

    images = np.array(images)
    return images, labels

In [8]:
bla = unique_sites["site"].iloc[:10]
test1, test2 = get_data(bla)
#print(test2.head())
print(test2.shape)

: 

In [ ]:
def train_model(images, label):
    model = keras.Sequential([
    Flatten(input_shape=(540, 960, 3)),
    Dense(8, activation="softmax")
    ])

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(images, label, epochs=2, batch_size = 32)
    return model

In [ ]:
model = train_model(test1, test2)
test3, test4 = get_data(["S0076","S0110","S0083","S0047"])

/Users/riael/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 128ms/step - accuracy: 0.1851 - loss: 179914.8906
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - accuracy: 0.4084 - loss: 47646.6367
(374, 11)


In [ ]:
def print_accuracy(model, images, labels):
    loss, acc = model.evaluate(images, labels)
    print(f"Model loss: {loss}, Model accuracy: {acc}")


In [ ]:
print_accuracy(model, test3, test4)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.2799 - loss: 34384.9727
Model loss: 33434.953125, Model accuracy: 0.2967914342880249
